In [217]:
import pandas as pd
import plotly.express as px
import os
import numpy as np

min_txs_per_day = 100

In [218]:
df = pd.read_csv('csv_exports/project_deploys_2022_12_09.csv')

df = df[ (df['txs_per_day'] >= min_txs_per_day)]
# df = df[ (df['type'] == 'Contract Project')]
df['year'] = df['launch_time'].str[:4]
df['date'] = pd.to_datetime( df['launch_time'].str[:10] )

df.sample(10)

,project,launch_time,num_contracts,type,num_transactions,txs_per_day,time_elapsed,year,date
174,Band Protocol,2021-12-14T11:32:09Z,52,Contract Project,45511.0,126.269962,360.426181,2021,2021-12-14
153,Zipswap,2021-12-10T18:50:39Z,133,Contract Project,71226.0,195.610442,364.121667,2021,2021-12-10
146,Kwenta,2022-02-12T00:34:04Z,36170,Contract Project,222772.0,740.393657,300.883183,2022,2022-02-12
197,Kyberswap,2022-06-16T07:54:02Z,67,Contract Project,20305.0,114.991903,176.577650,2022,2022-06-16
172,OP Horse Kingdom,2022-10-27T06:46:52Z,1,NFT Only Project,5405.0,123.898853,43.624294,2022,2022-10-27
215,Granary,2022-05-26T00:38:03Z,64,Contract Project,493262.0,2492.727721,197.880417,2022,2022-05-26
80,Lyra,2021-08-21T07:13:22Z,185,Contract Project,252379.0,530.647338,475.605891,2021,2021-08-21
137,Tarot Finance,2022-05-06T14:19:15Z,262,Contract Project,173284.0,797.404120,217.310139,2022,2022-05-06
67,Quix,2021-12-18T00:44:35Z,44,Contract Project,479918.0,1344.775670,356.875880,2021,2021-12-18
287,Layer Zero,2022-03-15T04:02:30Z,37,Contract Project,353041.0,1308.827186,269.738437,2022,2022-03-15


In [219]:
df['type'] = df['type'].str.replace('Contract Project','dApp')
df['type'] = df['type'].str.replace('NFT Only Project','NFT Project')
df['project'] = df['project'].str.replace(' Finance','')

In [220]:
exclude_list = ['Op','Optimistic Explorer']

df = df[~df['project'].isin(exclude_list)]

In [221]:
print(df.groupby(by='year').count())
df = df[df['year'] == '2022']
# display(df)

      project  launch_time  num_contracts  type  num_transactions  \
year                                                                
2021       25           25             25    25                25   
2022       55           55             55    55                55   

      txs_per_day  time_elapsed  date  
year                                   
2021           25            25    25  
2022           55            55    55  


In [222]:
fig = px.scatter(df, x="date", y="txs_per_day",
	        #  size="pop", 
                 color="type",
                 hover_name="project",
                #  size_max=60
                labels={"date":"Deployment Date", "txs_per_day":"Transactions per Day", "type":"App Type"}
                ,title = "Projects Deployed on Optimism in 2022 \
                        <br><sup>Plot Greater Than 100 Transactions per Day</sup>"
                , width=800, height=400
                 )
project_annotations = ['Velodrome','Pooltogether','Galxe','Aave','Curve','Beethoven X','Kwenta','Opensea','Atlantis World','Odos','Granary','Stargate']
for p in project_annotations:
        x_val = df[df['project'] == p]['date'].iloc[0]
        y_val = df[df['project'] == p]['txs_per_day'].iloc[0]
        fig.add_annotation(x= x_val, y= y_val,
                text=p,
                showarrow=True,
                arrowsize=1,
                ay= int( np.where(y_val > 2000, -12, 12) ),
                ax= int( np.where(x_val > pd.to_datetime("2022-03-15"), 12, -12) ),
                xref="x",
                yref="y",
                bgcolor="#E6ECF5",
                opacity=0.9,
                arrowcolor="#636363",
                font=dict(
                        size=10,
                        ),
                )
fig.show()
if not os.path.exists("img_outputs"):
        os.mkdir("img_outputs")
fig.write_image("img_outputs/op_deployments_2022.svg")
fig.write_image("img_outputs/op_deployments_2022.png")